In [1]:
import pandas as pd
import numpy as np
import re
import jieba
from tqdm import tqdm

In [2]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

# Load Brands Dictionary

In [3]:
# convert brandEN into Uppercase, remove spaces
brands = pd.read_csv('cleaned_brand.csv', header = 0, keep_default_na = False, index_col = 0)
brands['brandEN'] = np.array([str(brandEN).upper() for brandEN in brands['brandEN'].tolist()])
brands['brandCN'] = np.array([str(brandCN).replace(" ", "") for brandCN in brands['brandCN'].tolist()])
brands['brandEN'] = np.array([str(brandEN).replace(" ", "") for brandEN in brands['brandEN'].tolist()])
brands['brandCN'] = np.array([str(brandCN).replace("NA", "") for brandCN in brands['brandCN'].tolist()])
brands['brandEN'] = np.array([str(brandEN).replace("NA", "") for brandEN in brands['brandEN'].tolist()])

brands.head()
brands.tail()

# load brands needed to be removed 
rmv_brandslst = pd.read_excel('rmv_brands.xlsx', encoding = 'utf_8_sig', header = None)[0].tolist()
len(rmv_brandslst)

# filter brands
filtered_brandsdf = brands[brands['brandCN'].apply(lambda x: x not in rmv_brandslst) &
                           brands['brandEN'].apply(lambda x: x not in rmv_brandslst)]
filtered_brandsdf.to_csv('filtered_brands_dict.csv', encoding = 'utf_8_sig', header = None, index = False)

df = pd.read_csv('filtered_brands_with_alias.csv', header = None, names = ['brandCN','brandEN',"brandNN"])
df["brandID"] = range(len(df))
idx = 0
for nick_name in list(df['brandNN']):
    if not pd.isnull(nick_name):
        name_list = nick_name.split(', ')
        name_id = idx
        for name in name_list:
            if name.encode( 'UTF-8' ).isalpha():
                new_record = {'brandCN':np.nan,"brandEN":name,'brandNN':np.nan,'brandID':idx}
            else:
                new_record = {'brandCN':name,"brandEN":np.nan,'brandNN':np.nan,'brandID':idx}
            df = df.append(new_record,ignore_index = True)
    idx += 1
df.to_csv('cleaned_brand.csv',encoding = "utf_8_sig")

# combine brandCN, brandEN into a list
filtered_brandslst = df
filtered_name_dict = {}
for elem in range(len(filtered_brandslst)):
    if not pd.isnull(df['brandCN'][elem]):
        filtered_name_dict[len(filtered_name_dict)] = [df['brandCN'][elem],df['brandID'][elem]]
    if not pd.isnull(df['brandEN'][elem]):
        filtered_name_dict[len(filtered_name_dict)] = [df['brandEN'][elem],df['brandID'][elem]]    
filtered_brands_df = pd.DataFrame(data = filtered_name_dict).T
filtered_brands_df.columns = ['name','id']

# create a list for matching brands
pd.DataFrame(filtered_brands_df).to_csv('filtered_brands_df.csv', encoding = 'utf_8_sig', 
                                        header = None, index = False)
brand_names = filtered_brands_df['name'].tolist()

# prepare self-defined dictionary for brand names
userdict_brand = pd.DataFrame(brand_names)
userdict_brand[1] = 5000
userdict_brand.to_csv('userdict_brand.txt', sep = ' ', index = False, header = None)

,brandCN,brandEN,brandNN,brandID
0,奥蜜思,ORBIS,,0
1,阿芙,AFU,,1
2,伊蒂之屋,ETUDEHOUSE,爱丽小屋,2
3,艾诗缇,ASTALIFT,,3
4,嫒碧知,EPISTEME,,4


,brandCN,brandEN,brandNN,brandID
3449,,YES,,3187
3450,许三多,,,3223
3451,三多家,,,3223
3452,红腰子,,,3223
3453,水之密语,,,3300


67

# Load End Words Dictionary

In [4]:
# load end words list
end_words = pd.read_csv('end_words.csv', encoding = 'utf_8_sig', header = None)[0].tolist()
len(end_words)
pd.DataFrame(end_words).head()

# prepare self-defined dictionary for end words
userdict_endword = pd.DataFrame(end_words)
userdict_endword[1] = 10000
userdict_endword.to_csv('userdict_endword.txt', sep = ' ', index = False, header = None)

94

,0
0,啫喱
1,磨砂膏
2,润肤霜
3,润肤露
4,身体乳


# Load Stop Words Dictionary

In [5]:
stop_words = pd.read_excel('stop_words.xlsx', encoding = 'utf_8_sig', header = None)[0].tolist()
len(stop_words)
pd.DataFrame(stop_words).head()

113

,0
0,滴
1,的
2,系列的
3,牌
4,版


# Product Identification

<b>`jieba`</b>

In [6]:
'豆腐' in brand_names

True

In [7]:
jieba.load_userdict('userdict_brand.txt')
jieba.load_userdict('userdict_endword.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/09/s99ydtzd3g92s37wkq1q5dfw0000gn/T/jieba.cache
Loading model cost 1.375 seconds.
Prefix dict has been built successfully.


In [8]:
def remove_html(text):
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = text.strip('')
    return text

In [9]:
def remove_special_chars(text):
    text = re.sub(r'[^a-zA-Z0-9\u4e00-\u9fff]' , '', text)
    return text

In [10]:
def remove_stop_words(text):
    for i in stop_words:
        text = text.replace(i,'')
    return text   

In [11]:
def normalizer(text):
    text = remove_html(text)
    text = remove_special_chars(text)
    text = remove_stop_words(text) 
    text = text.upper()
    text = re.sub(' +', ' ', text)
    return text

In [12]:
def get_products_name(post):
    
    # initialize a set to record the row number in the brand name dictionary
    idx_set = set()
    
    # initialize a lost to record identified products
    products = []
    
    # text pre-processing
    post = normalizer(post)
    
    # words tokenization
    tokens = jieba.lcut(post, cut_all = False, HMM = True)
    
    # get the index of brand name, end word
    count = 0
    brand_idx = []
    endword_idx = []
    for i in tokens:
        if i in brand_names:
            brand_idx.append(count)
        elif i in end_words:
            endword_idx.append(count)
        count += 1
    
    # extract product name
    for i in brand_idx:
        for j in endword_idx: 
            if j>=i and j<i+5:
                idx_set.add(
                    filtered_brands_df['id'][filtered_brands_df[filtered_brands_df['name'] == tokens[i]].index.tolist()[0]]
                )
                products.append(''.join(tokens[i:j+1]))
    
    # remove duplicated products
    if len(idx_set) == 1:        
        delete = 0
        for i in range(len(products)):
            ti = i - delete
            for j in range(len(products)):
                if ti == j:
                    continue
    #             if products[ti] in products[j]:
                if all(elem in list(products[ti]) for elem in list(products[j])):
                    del(products[ti])
                    delete += 1
                    break
        brand_id = list(idx_set)[0]
    else:
        products = []
        brand_id = ""
        
    return products, brand_id

In [17]:
def mkdir(path):
    import os
    
#     path = path.strip()
#     path = path.rstrip("\\")
    isExists = os.path.exists(path)
 
    if not isExists:
        os.makedirs(path) 
        print(path + '\nCreate Successfully!')
        return True
    else:
        print(path + '\nAlready Exist!')
        return False

In [19]:
for j in tqdm(range(14)):
    
    uid_list = list(pd.read_csv('final_uid_list/final_uid_list' + str(j) + '.csv', header = None)[1])
    
    # create a new folder
    mkpath = "product_name/" + str(j)
    mkdir(mkpath)
    
    for uid in tqdm(uid_list):

        matches = []

        # posts file path
        inpath = 'post/' + str(j) + '/' + str(uid) + '/'+ str(uid) + '.csv'
        posts = pd.read_csv(inpath)

        # append a column for product
        header = posts.columns.values.tolist()
        header.append('产品')
        header.append('品牌')


        for i in range(len(posts)):
            # extract post
            post = posts['正文'][i]

            # identify product name
            try:
                products, brand_id = get_products_name(post)
            except Exception as e:
                print('Error: ', e,'\n',uid, i+2) # i+2: row number of error in the excel file

            # record identified product name
            if len(products) != 0:
                products = ", ".join(products)
                match = list(posts.iloc[i])
                match.append(products)
                match.append(brand_id)
                matches.append(match)

        # append product name column to the data frame
        matches = pd.DataFrame(matches, columns = header)
        
        outpath = mkpath + '/' + str(uid) + '_prodname.csv'
        matches.to_csv(outpath, index = 0, encoding = 'utf_8_sig')






  0%|          | 0/14 [00:00<?, ?it/s]

product_name/0
Create Successfully!


True







  0%|          | 0/199 [00:00<?, ?it/s]





  1%|          | 1/199 [00:04<14:49,  4.49s/it]





  1%|          | 2/199 [00:05<10:57,  3.34s/it]





  2%|▏         | 3/199 [00:06<08:32,  2.61s/it]





  2%|▏         | 4/199 [00:09<09:11,  2.83s/it]





  3%|▎         | 5/199 [00:10<07:27,  2.31s/it]





  3%|▎         | 6/199 [00:10<05:26,  1.69s/it]





  4%|▎         | 7/199 [00:15<08:04,  2.52s/it]





  4%|▍         | 8/199 [00:16<07:10,  2.25s/it]





  5%|▍         | 9/199 [00:17<05:31,  1.75s/it]





  5%|▌         | 10/199 [00:18<04:49,  1.53s/it]





  6%|▌         | 11/199 [00:19<04:22,  1.39s/it]





  6%|▌         | 12/199 [00:19<03:15,  1.05s/it]





  7%|▋         | 13/199 [00:22<04:25,  1.43s/it]





  7%|▋         | 14/199 [00:26<07:08,  2.32s/it]





  8%|▊         | 15/199 [00:28<06:46,  2.21s/it]





  8%|▊         | 16/199 [00:28<04:59,  1.64s/it]





  9%|▊         | 17/199 [00:29<04:28,  1.47s/it]





  9%|▉         | 18/199 [00:31<04:42,  1

product_name/1
Create Successfully!


...

True







  0%|          | 0/195 [00:00<?, ?it/s]





  1%|          | 1/195 [00:01<04:24,  1.36s/it]





  1%|          | 2/195 [00:02<03:46,  1.17s/it]





  2%|▏         | 4/195 [00:02<02:41,  1.19it/s]





  3%|▎         | 6/195 [00:02<02:09,  1.46it/s]





  4%|▎         | 7/195 [00:03<01:43,  1.81it/s]





  4%|▍         | 8/195 [00:05<03:04,  1.01it/s]





  5%|▍         | 9/195 [00:07<04:25,  1.43s/it]





  5%|▌         | 10/195 [00:08<04:10,  1.35s/it]





  6%|▌         | 12/195 [00:10<03:30,  1.15s/it]





  7%|▋         | 13/195 [00:11<03:17,  1.08s/it]





  7%|▋         | 14/195 [00:11<02:50,  1.06it/s]





  8%|▊         | 15/195 [00:12<02:36,  1.15it/s]





  9%|▊         | 17/195 [00:13<02:24,  1.23it/s]





  9%|▉         | 18/195 [00:17<05:11,  1.76s/it]





 10%|▉         | 19/195 [00:18<04:11,  1.43s/it]





 10%|█         | 20/195 [00:18<03:02,  1.04s/it]





 11%|█▏        | 22/195 [00:19<02:23,  1.20it/s]





 12%|█▏        | 23/195 [00:20<02:36, 

product_name/13
Create Successfully!


True







  0%|          | 0/178 [00:00<?, ?it/s]





  1%|          | 1/178 [00:01<03:02,  1.03s/it]





  1%|          | 2/178 [00:02<03:01,  1.03s/it]





  2%|▏         | 3/178 [00:02<02:41,  1.09it/s]





  2%|▏         | 4/178 [00:04<02:58,  1.03s/it]





  3%|▎         | 5/178 [00:04<02:38,  1.09it/s]





  3%|▎         | 6/178 [00:05<02:43,  1.05it/s]





  4%|▍         | 7/178 [00:07<03:19,  1.16s/it]





  4%|▍         | 8/178 [00:07<02:45,  1.03it/s]





  5%|▌         | 9/178 [00:09<03:39,  1.30s/it]





  6%|▌         | 10/178 [00:10<02:55,  1.04s/it]





  6%|▌         | 11/178 [00:11<03:20,  1.20s/it]





  7%|▋         | 12/178 [00:12<03:06,  1.13s/it]





  8%|▊         | 14/178 [00:13<02:32,  1.08it/s]





  8%|▊         | 15/178 [00:14<02:40,  1.01it/s]





  9%|▉         | 16/178 [00:16<02:59,  1.11s/it]





 10%|▉         | 17/178 [00:17<02:50,  1.06s/it]





 10%|█         | 18/178 [00:17<02:21,  1.13it/s]





 11%|█         | 19/178 [00:20<03:52,  1